In [ ]:
import pdfplumber
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import quote
import time
import random
import os
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [ ]:
def extraer_texto_limpio(ruta_pdf, margen_superior=60, margen_inferior=60):
    """
    Extrae el texto completo de un PDF eliminando encabezados y pies de página, recortando por posición física en el PDF.
    Devuelve también las páginas recortadas como objetos pdfplumber.Page.

    Parámetros:
        ruta_pdf (str): Ruta del archivo PDF.
        margen_superior (int): Altura en puntos a recortar desde la parte superior.
        margen_inferior (int): Altura en puntos a recortar desde la parte inferior.

    Devuelve:
        tuple[str, list]: Texto limpio y lista de páginas recortadas (pdfplumber.Page)
    """
    texto_limpio = ""
    paginas_recortadas = []

    with pdfplumber.open(ruta_pdf) as pdf:
        for page in pdf.pages:
            y0, y1 = margen_inferior, page.height - margen_superior
            area_util = (0, y0, page.width, y1)
            recorte = page.within_bbox(area_util)
            paginas_recortadas.append(recorte)
            texto = recorte.extract_text()
            if texto:
                texto_limpio += texto + "\n"

    return texto_limpio, paginas_recortadas

def extraer_seccion(ruta_pdf, titulo=None, inicio=None, fin=None):
    """
    Extrae una sección genérica de una guía en PDF usando pdfplumber.

    Parámetros:
        ruta_pdf (str): Ruta del archivo PDF.
        titulo (str): Título principal de la sección (ej. 'Descripción de la asignatura y temario').
        inicio (str): Subtítulo o punto de inicio del bloque (ej. 'Descripción de la asignatura').
        fin (str): Subtítulo o punto final del bloque (ej. 'Temario de la asignatura').

    Devuelve:
        str: Texto extraído entre los límites indicados, o el texto más cercano posible si faltan.
    """
    _, paginas_recortadas = extraer_texto_limpio(ruta_pdf) # Importante tener la función de eliminar encabezados y pies de página

    texto_a_buscar = ""
    for page in paginas_recortadas[2:]:  # omitimos portada e índice
        texto = page.extract_text()
        if texto:
            texto_a_buscar += texto + "\n"

    # Si no se pasa nada, devolvemos todo el texto limpio
    if not any([titulo, inicio, fin]):
        return texto_a_buscar.strip()

    def patron_dinamico(texto, es_subtitulo=False):
        """
        Genera un patrón regex flexible con numeración opcional (4., 5.1., etc.)
        """
        if not texto:
            return None
        if es_subtitulo:
            return rf"\b\d+\.\d+\.\s*{re.escape(texto)}\b"
        else:
            return rf"\b\d+\.\s*{re.escape(texto)}\b"

    patron_titulo = patron_dinamico(titulo)
    patron_inicio = patron_dinamico(inicio, es_subtitulo=True)
    patron_fin = patron_dinamico(fin, es_subtitulo=True)

    # Paso 1: localizar el título principal si existe
    texto_post_titulo = texto_a_buscar
    if patron_titulo:
        match_titulo = re.search(patron_titulo, texto_a_buscar, flags=re.IGNORECASE)
        if match_titulo:
            texto_post_titulo = texto_a_buscar[match_titulo.end():]

    # Paso 2: buscar el inicio y fin dentro del texto posterior al título
    match_inicio = re.search(patron_inicio, texto_post_titulo, flags=re.IGNORECASE) if patron_inicio else None
    match_fin = re.search(patron_fin, texto_post_titulo, flags=re.IGNORECASE) if patron_fin else None

    # Casos posibles
    if match_inicio and match_fin:
        texto_extraido = texto_post_titulo[match_inicio.end():match_fin.start()]
    elif match_inicio and not match_fin:
        texto_extraido = texto_post_titulo[match_inicio.end():]
    elif not match_inicio and patron_titulo:
        texto_extraido = texto_post_titulo
    else:
        # Si no encuentra nada, devolvemos algo razonable
        texto_extraido = texto_a_buscar

    return texto_extraido.strip()


def extraer_competencias_resultados(texto, model):
    """
    Extrae competencias de un texto en formato:
        '\nCodigo - Texto de la competencia'
    
    Parámetros:
        texto (str): Texto completo con las competencias.
    
    Retorna:
        list[dict]: Lista de competencias con campos 'codigo' y 'texto'.
    """
    codigos = re.findall(r'\n?(.*?)\s*-', texto)
    
    textos = re.split(r'\n.*?-', texto)
    textos[0] = textos[0].split('-')[1]
    textos = [t.replace('\n', ' ').strip() for t in textos if t.strip()]
    
    competencias = []
    
    for i, codigo in enumerate(codigos):
        texto_competencia = textos[i] if i < len(textos) else ""
        vector = model.encode(texto_competencia)       
        competencias.append({
            "codigo": codigo.strip(),
            "texto": texto_competencia,
            "vector": vector.tolist() # 512 dimensiones
        })
    
    return competencias

def bulk_index_data(es, data, index_name):
    batch_size = 50  
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for doc in batch:
            actions.append({
                "_index": index_name,
                "_id": doc['id_asignatura'],
                "_source": doc
            })
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

## Ejemplo

In [ ]:
ruta = 'Guias Docentes/1_GA_61CI_615000215_1S_2025-26.pdf'
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

texto_competencias = extraer_seccion(
    ruta,
    titulo = "Competencias y resultados de aprendizaje",
    inicio = "Competencias",
    fin = "Resultados del aprendizaje"
)
competencias = extraer_competencias_resultados(texto_competencias, model)
texto_resultados = extraer_seccion(
    ruta,
    titulo = "Competencias y resultados de aprendizaje",
    inicio = "Resultados del aprendizaje",
    fin = "Descripción de la asignatura y temario"
)
resultados = extraer_competencias_resultados(texto_resultados, model)


In [ ]:
es = Elasticsearch("http://elasticsearch:9200")

index_name = "asignaturas"

mapping = {
    "mappings": {
        "properties": {
            "id_asignatura": {"type": "keyword"},
            "competencias": {
                "type": "nested",
                "properties": {
                    "codigo": {"type": "keyword"},
                    "texto": {"type": "text"},
                    "vector": {"type": "dense_vector", "dims": 512, "similarity": "cosine"}  
                }
            },
            "resultados": {
                "type": "nested",
                "properties": {
                    "codigo": {"type": "keyword"},
                    "texto": {"type": "text"},
                    "vector": {"type": "dense_vector", "dims": 512, "similarity": "cosine"}  
                }
            }
        }
    }
}

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)

documento = {
    "id_asignatura": 1,
    "competencias": competencias,
    "resultados": resultados
}

es.index(index=index_name, id=1, body=documento)

## General

In [ ]:
directory = "Guias Docentes"
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

es = Elasticsearch("http://elasticsearch:9200")

index_name = "asignaturas"

mapping = {
    "mappings": {
        "properties": {
            "id_asignatura": {"type": "keyword"},
            "competencias": {
                "type": "nested",
                "properties": {
                    "codigo": {"type": "keyword"},
                    "texto": {"type": "text"},
                    "vector": {"type": "dense_vector", "dims": 512, "similarity": "cosine"}  
                }
            },
            "resultados": {
                "type": "nested",
                "properties": {
                    "codigo": {"type": "keyword"},
                    "texto": {"type": "text"},
                    "vector": {"type": "dense_vector", "dims": 512, "similarity": "cosine"}  
                }
            }
        }
    }
}

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)

documentos = []
i = 0
for file in os.listdir(directory):
    if file.endswith(".pdf"):  
        i += 1
        ruta = os.path.join(directory, file)
        texto_competencias = extraer_seccion(
            ruta,
            titulo = "Competencias y resultados de aprendizaje",
            inicio = "Competencias",
            fin = "Resultados del aprendizaje"
        )
        competencias = extraer_competencias_resultados(texto_competencias, model)
        texto_resultados = extraer_seccion(
            ruta,
            titulo = "Competencias y resultados de aprendizaje",
            inicio = "Resultados del aprendizaje",
            fin = "Descripción de la asignatura y temario"
        )
        resultados = extraer_competencias_resultados(texto_resultados, model)
        documento = {
            "id_asignatura": i,
            "competencias": competencias,
            "resultados": resultados
        }
        documentos.append(documento)

bulk_index_data(es, documentos, index_name)